In [2]:
import json

In [3]:
from mpcontribs.client import Client
client = Client(apikey="GdU0AaMi4XS8vSb2YOsMRRyEgnD3quCD", project="screening_inorganic_pv")

/home/matthew/miniforge3/envs/tlopt/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
query = {}
fields = ['identifier', 'id', 'formula', 'tables','data']

In [5]:
data = client.query_contributions(query=query,fields=fields,paginate=True)

In [6]:
all = data['data']

In [7]:
len(all)

727

In [8]:
all[0]

{'identifier': 'mp-27628',
 'id': '600b713404f51b4a4e905f68',
 'formula': 'Te3Cl2',
 'tables': [{'id': '600b713304f51b4a4e905f66',
   'name': 'absorption',
   'md5': 'd9c9edd99c27913b8e65af0a0db46dee',
   'attrs': {'title': 'optical absorption spectrum',
    'labels': {'index': 'hν [eV]', 'value': 'α [cm⁻¹]', 'variable': ''}},
   'columns': ['α [cm⁻¹]'],
   'total_data_rows': 684,
   'total_data_pages': 35},
  {'id': '600b713404f51b4a4e905f67',
   'name': 'DOS',
   'md5': '3c436b39ae9275b7af94301ba5668035',
   'attrs': {'title': 'electronic density of states',
    'labels': {'index': 'E [eV]', 'value': 'DOS [eV⁻¹]', 'variable': ''}},
   'columns': ['DOS [eV⁻¹]'],
   'total_data_rows': 5000,
   'total_data_pages': 250}],
 'data': {'SLME': {'500nm': {'display': '28.7058 %',
    'value': 28.7058,
    'error': None,
    'unit': '%'},
   '1000nm': {'display': '28.7088 %',
    'value': 28.7088,
    'error': None,
    'unit': '%'}},
  'ΔE': {'corrected': {'display': '1.60729 eV',
    'value':

In [9]:
all[0]['data'].keys()

dict_keys(['SLME', 'ΔE', 'mᵉ', 'mʰ'])

In [10]:
mpid_to_SLME = {}
mpid_to_gap = {}
gap_data = {}
for mat in all:
    mpid = mat['identifier']
    dat = mat['data']
    mpid_to_SLME[mpid]=dat['SLME']['500nm']['value']
    fun_gap = dat['ΔE']['corrected']['value']
    dir_gap = dat['ΔE']['direct']['value']
    da_gap = dat['ΔE']['dipole']['value']
    offset = da_gap - fun_gap
    mpid_to_gap[mpid]=fun_gap
    gap_data[mpid] = {"fun_gap": fun_gap, "dir_gap": dir_gap, "da_gap": da_gap, "offset": offset}

In [11]:
len(mpid_to_SLME)

727

In [12]:
len(mpid_to_gap)

727

In [13]:
with open("fab_SLMEs.json","w+") as outfile:
    json.dump(mpid_to_SLME,outfile)
with open("fab_gaps.json","w+") as outfile:
    json.dump(mpid_to_gap,outfile)
with open("fab_gap_data.json","w+") as outfile:
    json.dump(gap_data, outfile)

In [14]:
mpids = []
ids = []
forms = []
for i in all:
    mpids.append(i['identifier'])
    ids.append(i['tables'][0]['id'])
    forms.append(i['formula'])
    

In [15]:
import pandas as pd
df_data = {"mpid": mpids, "id": ids, "formula": forms}
df = pd.DataFrame.from_dict(df_data)

In [16]:
pd.DataFrame.to_csv(df,"fabini_labels.csv",index=False)

In [17]:
dos_dict = {}
for i in all:
    mpid = i['identifier']
    id = i['tables'][1]['id']
    dos_dict[id]=mpid

In [18]:
with open("dos_id_to_mpid.json","w+") as outfile:
    json.dump(dos_dict,outfile)

In [19]:
data = client.download_contributions(query=query,include=["tables"])

100%|##########| 162/162 [02:19<00:00,  1.17it/s]


request failed with status 400!
request failed with status 400!
request failed with status 400!
request failed with status 400!
request failed with status 400!


---------------------------------------------------------------------------
KeyError                                  Traceback (most recent call last)
Cell In[19], line 1
----> 1 data = client.download_contributions(query=query,include=["tables"])

File ~/miniforge3/envs/tlopt/lib/python3.10/site-packages/mpcontribs/client/__init__.py:2257, in Client.download_contributions(self, query, outdir, overwrite, include, timeout)
   2254 if not cids:
   2255     continue
-> 2257 paths = self._download_resource(
   2258     resource="contributions", ids=cids, fmt=fmt,
   2259     outdir=outdir, overwrite=overwrite, timeout=timeout
   2260 )
   2261 logger.debug(
   2262     f"Downloaded {len(cids)} contributions for '{name}' in {len(paths)} file(s)."
   2263 )
   2265 for path in paths:

File ~/miniforge3/envs/tlopt/lib/python3.10/site-packages/mpcontribs/client/__init__.py:2411, in Client._download_resource(self, resource, ids, outdir, overwrite, timeout, fmt)
   2408     responses = _run_fut